# Workout: Async Patterns

## Drill 1: Async Endpoint 🟢
**Task:** Convert this sync endpoint to async

In [ ]:
import httpx

# Sync version
# @app.get("/data")
# def get_data():
#     response = requests.get("https://api.example.com/data")
#     return response.json()

# Async version
# @app.get("/data")
async def get_data():
    async with httpx.AsyncClient() as client:
        response = await client.get("https://api.example.com/data")
        return response.json()

---
## Drill 2: httpx Client 🟢
**Task:** Make an async HTTP request with httpx

In [ ]:
import httpx

async def fetch_user(user_id: int):
    # Use async with httpx.AsyncClient
    # GET https://api.example.com/users/{user_id}
    async with httpx.AsyncClient() as client:
        response = await client.get(f"https://api.example.com/users/{user_id}")
        return response.json()

---
## Drill 3: Concurrent Requests 🟡
**Task:** Fetch 5 URLs concurrently

In [ ]:
import asyncio
import httpx

urls = [
    "https://api.example.com/1",
    "https://api.example.com/2",
    "https://api.example.com/3",
    "https://api.example.com/4",
    "https://api.example.com/5",
]

async def fetch_all(urls: list[str]):
    # Use asyncio.gather to fetch all concurrently
    async with httpx.AsyncClient() as client:
        tasks = [client.get(url) for url in urls]
        responses = await asyncio.gather(*tasks)
        return [r.json() for r in responses]

---
## Drill 4: Error Handling with gather 🟡
**Task:** Handle errors in concurrent requests

In [ ]:
async def fetch_all_safe(urls: list[str]):
    # Use return_exceptions=True
    # Return results or error message for each URL
    async with httpx.AsyncClient() as client:
        tasks = [client.get(url) for url in urls]
        results = await asyncio.gather(*tasks, return_exceptions=True)
        return results

---
## Drill 5: Semaphore 🟡
**Task:** Limit concurrent requests to 10

In [ ]:
import asyncio

semaphore = asyncio.Semaphore(10)

async def fetch_with_limit(client, url):
    async with semaphore:
        response = await client.get(url)
        return response.json()

---
## Drill 6: Reusable Client 🔴
**Task:** Create a shared client with lifespan

In [ ]:
from contextlib import asynccontextmanager
import httpx

@asynccontextmanager
async def lifespan(app):
    # Startup: create httpx.AsyncClient
    app.state.http_client = httpx.AsyncClient()
    yield
    # Shutdown: close client
    await app.state.http_client.aclose()

---
## Drill 7: Background Tasks 🟢
**Task:** Send email in background after user creation

In [ ]:
from fastapi import BackgroundTasks

def send_email(email: str, subject: str):
    # Simulate sending
    print(f"Sending {subject} to {email}")

# @app.post("/users")
def create_user(user, background_tasks: BackgroundTasks):
    # Create user
    # Add background task to send welcome email
    background_tasks.add_task(send_email, user.email, "Welcome!")
    return user

---
## Drill 8: Timeout 🟡
**Task:** Add 5 second timeout to HTTP request

In [ ]:
import asyncio
from fastapi import HTTPException

async def fetch_with_timeout(url: str):
    try:
        async with httpx.AsyncClient(timeout=5.0) as client:
            return await client.get(url)
    except httpx.TimeoutException:
        raise HTTPException(status_code=504, detail="Gateway timeout")

---
## Drill 9: Spot the Bug 🟡
**Task:** What's wrong with this code?

In [ ]:
# @app.get("/data")
# async def get_data():
#     response = requests.get("https://api.example.com")  # Bug?
#     return response.json()

# Answer: Using sync `requests` in async function blocks the event loop!
# Should use `httpx.AsyncClient` instead

---
## Drill 10: When to Use Async 🟢
**Task:** Mark each as sync or async:

In [ ]:
# A. Calling external API         -> async
# B. Heavy computation (math)     -> sync (use run_in_executor)
# C. Database query (async driver)-> async
# D. Simple dict lookup           -> sync
# E. Multiple concurrent I/O      -> async

---
## Self-Check

- [ ] Can convert sync to async
- [ ] Can use httpx for HTTP requests
- [ ] Can run concurrent requests
- [ ] Know when to use async vs sync